In [1]:
import numpy as np
import pandas as pd

In [2]:
listing_df = pd.read_csv('C:\\DataScience\\00_AnalysisProjects\\AirbnbAnalysis\\Data\\London_listings.csv')
listing_df.shape   # (77096, 96)

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (61,62,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(77096, 96)

In [3]:
calendar_df = pd.read_csv('C:\\DataScience\\00_AnalysisProjects\\AirbnbAnalysis\\Data\\London_calendar.csv')
calendar_df.shape 

(28139675, 4)

In [4]:
calendar = calendar_df
calendar["price"] = calendar["price"].apply(lambda x: str(x).replace("$", ""))
calendar["price"] = pd.to_numeric(calendar["price"] , errors="coerce")
df1  = calendar.groupby("date")[["price"]].sum()
df1["mean"]  = calendar.groupby("date")[["price"]].mean()
df1.columns = ["Total", "Average"]
df1.head()

,Total,Average
date,,
2018-12-07,686632.0,145.411266
2018-12-08,1191258.0,131.094751
2018-12-09,2021083.0,117.457023
2018-12-10,2326273.0,116.968675
2018-12-11,2400023.0,116.687233


In [5]:
df2 = calendar.set_index("date")
df2.index = pd.to_datetime(df2.index)
df2 =  df2[["price"]].resample("M").mean()
df2.head()

,price
date,
2018-12-31,122.479159
2019-01-31,114.149572
2019-02-28,113.226533
2019-03-31,120.287040
2019-04-30,126.576012


In [6]:
import plotly as py
from plotly.offline import iplot, plot, init_notebook_mode, download_plotlyjs
import plotly.graph_objs as go
init_notebook_mode(connected=True)
import plotly.offline as offline

In [7]:
trace1 = go.Scatter(
    x = df1.index,
    y = df1["Total"]
)
data = [trace1]
layout = go.Layout(
    title = "Price by each time",
    xaxis  = dict(title = "Time"),
    yaxis = dict(title = "Total ($)")
)
trace2 = go.Scatter(
    x = df1.index,
    y = df1["Average"]
)

data2 = [trace2]
layout2 = go.Layout(
    title = "Price by each time",
    xaxis  = dict(title = "Time"),
    yaxis = dict(title = "Mean ($)")
)
fig = go.Figure(data = data, layout = layout)
fig2 = go.Figure(data = data2, layout = layout2)
offline.iplot(fig)

In [8]:
offline.iplot(fig2)

In [9]:
trace3 = go.Scatter(
    x = df2.index[:-1],
    y = df2.price[:-1]
)
layout3 = go.Layout(
    title = "Average price by month",
    xaxis = dict(title = "time"),
    yaxis = dict(title = "Price")
)
data3 = [trace3]
fig3 = go.Figure(data= data3, layout= layout3)
offline.iplot(fig3)

In [10]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [11]:
def draw_interactive_graph(mode):
    df1.index = pd.to_datetime(df1.index)
    decomposition = seasonal_decompose(df1[[mode]])
    trace4_1 = go.Scatter(
        x = decomposition.observed.index, 
        y = decomposition.observed[mode],
        name = "Observed"
    )
    trace4_2 = go.Scatter(
        x = decomposition.trend.index,
        y = decomposition.trend[mode],
        name = "Trend"
    )
    trace4_3 = go.Scatter(
        x = decomposition.seasonal.index,
        y = decomposition.seasonal[mode],
        name = "Seasonal"
    )
    trace4_4 = go.Scatter(
        x = decomposition.resid.index,
        y = decomposition.resid[mode],
        name = "Resid"
    )

    fig = py.tools.make_subplots(rows=4, cols=1, subplot_titles=('Observed', 'Trend',
                                                              'Seasonal', 'Residiual'))
    # append trace into fig
    fig.append_trace(trace4_1, 1, 1)
    fig.append_trace(trace4_2, 2, 1)
    fig.append_trace(trace4_3, 3, 1)
    fig.append_trace(trace4_4, 4, 1)

    fig['layout'].update( title='Descompose with TimeSeri')
    offline.iplot(fig)

In [12]:
draw_interactive_graph("Average")

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]



In [13]:
draw_interactive_graph("Total")

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]



In [15]:
def loc_city(x):
    if "," not in str(x):
        return x
    if "live" in str(x) or "Next door to" in str(x) or "live" in str(x) or "having" in str(x):
        return "USA"
    return str(x).split(",")[0]
a = listing_df["host_location"].apply(lambda x: loc_city(x))

In [17]:
df_listing = listing_df[listing_df.applymap(np.isreal)]
df_listing.dropna(how = "all", axis = 1, inplace = True)
df_listing.head()
df_listing["City"]  = a
df_listing.head(1)

,id,scrape_id,host_id,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,...,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,calculated_host_listings_count,reviews_per_month,City
0,9554,20181207034825,31655,4.0,4.0,51.587767,-0.105666,2,NaN,1.0,...,10.0,9.0,10.0,10.0,9.0,10.0,NaN,4,1.65,London


In [18]:
df_seattle = df_listing[df_listing["City"] == "Seattle"]
df_seattle.head()

,id,scrape_id,host_id,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,...,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,calculated_host_listings_count,reviews_per_month,City
9804,6550729,20181207034825,4271330,1.0,1.0,51.538811,-0.045712,2,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.07,Seattle
24803,14279479,20181207034825,20786100,2.0,2.0,51.563350,-0.123351,4,1.0,1.0,...,10.0,10.0,6.0,8.0,8.0,9.0,NaN,2,0.11,Seattle
25497,14644267,20181207034825,17597728,2.0,2.0,51.485301,-0.073172,2,1.0,1.0,...,9.0,9.0,10.0,10.0,9.0,9.0,NaN,2,1.17,Seattle
37104,19351472,20181207034825,20786100,2.0,2.0,51.561813,-0.123615,2,1.0,1.0,...,10.0,9.0,10.0,10.0,10.0,10.0,NaN,2,0.60,Seattle
69921,29114534,20181207034825,17597728,2.0,2.0,51.485174,-0.073388,2,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,Seattle


In [19]:
calendar_clean = calendar.dropna()
calendar_clean.set_index("date", inplace = True)
calendar_clean.head()

,listing_id,available,price
date,,,
2019-11-25,9554,t,35.0
2019-11-24,9554,t,35.0
2019-11-23,9554,t,39.0
2019-11-22,9554,t,39.0
2019-11-21,9554,t,35.0


In [20]:
calendar_clean.index = pd.to_datetime(calendar_clean.index)
number_hire_room = calendar_clean.resample("M")[["price"]].count()
total_price_each_month  = calendar_clean.resample("M")[["price"]].sum()

In [21]:
trace5 = go.Scatter(
    x = number_hire_room.index[:-1],
    y = number_hire_room.price[:-1]
)
data5 = [trace5]
layout5 = go.Layout(
    title = "Number of Hire Room by Month in Seattle",
    xaxis = dict(title = "Month"),
    yaxis = dict(title = "Number hirde")
)
fig5  = go.Figure(data = data5, layout = layout5)

In [22]:
trace6 = go.Scatter(
    x = number_hire_room.index[:-1],
    y = number_hire_room.price[:-1]/number_hire_room.price[0]
)
data6 = [trace6]
layout6 = go.Layout(
    title = "the ratio of the number of rooms compare with the first month",
    xaxis = dict(title = "Month"),
    yaxis = dict(title = "Ratio")
)
fig6 = go.Figure(data = data6, layout = layout6)

In [23]:
offline.iplot(fig5)

In [24]:
offline.iplot(fig6)

In [25]:
from scipy import stats

In [26]:
a = calendar_clean.index.month
# calendar_clean["Month"] = a
calendar_clean = calendar_clean.assign(Month = a)
calendar_clean.head()

,listing_id,available,price,Month
date,,,,
2019-11-25,9554,t,35.0,11
2019-11-24,9554,t,35.0,11
2019-11-23,9554,t,39.0,11
2019-11-22,9554,t,39.0,11
2019-11-21,9554,t,35.0,11


In [27]:


result = []
for i in range(1,13):
    result.append(np.array([calendar_clean[calendar_clean["Month"] == i].price]))



In [28]:
data_score = []
for i in range(11):
    score = stats.ttest_rel(result[i][0][:64911],result[-1][0][:64911])
    data_score.append((score[0], score[1]))

In [29]:
score_board = pd.DataFrame(data = data_score, columns = ["Test Statistic", "P_value"])
score_board["Month"] = range(1, 12)
score_board.set_index("Month", inplace = True)
score_board

,Test Statistic,P_value
Month,,
1,-14.907433,3.571048e-50
2,-15.748268,8.957391e-56
3,-14.036928,1.077138e-44
4,-10.806497,3.382590e-27
5,-11.480491,1.769421e-30
6,-6.952102,3.632667e-12
7,-2.532751,1.131948e-02
8,-0.698523,4.848527e-01
9,-5.631242,1.796535e-08


In [30]:
offline.iplot(fig3)